# Terra Signal Hackathon
This notebook is provided as a starting point. Feel free to use it, discard it, modify it, or pretend it doesn't exist.

In [0]:
%pip install pandas

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file using pandas
file_path = "./history.csv"
df = pd.read_csv(file_path)
df.head().transpose()

Análise Exploratória e limpeza dos dados

In [0]:
df.info()

In [0]:
for col in df.columns:
    print(f"\n--- {col} ---")
    print(df[col].value_counts())


In [0]:
# removendo coluna irrelevante
df_clean = df.copy()
df_clean = df_clean.drop(columns=["customerID"])

# limpando a coluna 'tenure'

# substituindo 'unknown' por NaN e convertendo para numérico
df_clean["tenure"] = df_clean["tenure"].replace("unknown", pd.NA)
df_clean["tenure"] = pd.to_numeric(df_clean["tenure"], errors="coerce")

# substituindo valores 0 por NaN
df_clean.loc[df_clean["tenure"] == 0, "tenure"] = pd.NA

# preenchendo NaN com a mediana e convertendo para inteiro
df_clean["tenure"] = df_clean["tenure"].fillna(df_clean["tenure"].median())
df_clean["tenure"] = df_clean["tenure"].astype(int)

# normalizando phone service
df_clean["PhoneService"] = (
    df_clean["PhoneService"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"yes": 1, "no": 0})
    .astype(int)  # <- evitar FutureWarning
)

# normalizando multiple lines
df_clean["MultipleLines"] = (
    df_clean["MultipleLines"]
    .replace({"No phone service": "No"})
    .map({"Yes": 1, "No": 0})
    .astype(int)
)

# normalizando colunas de internet
internet_cols = [
    "OnlineSecurity", "OnlineBackup", "DeviceProtection",
    "TechSupport", "StreamingTV", "StreamingMovies"
]

for col in internet_cols:
    df_clean[col] = (
        df_clean[col]
        .replace({"No internet service": "No"})
        .map({"Yes": 1, "No": 0})
        .astype(int)
    )

# normalizando colunas binárias
for col in ["Partner", "Dependents", "PaperlessBilling"]:
    df_clean[col] = df_clean[col].map({"Yes": 1, "No": 0}).astype(int)

# convertendo total charges para numérico e tratando NaN
df_clean["TotalCharges"] = pd.to_numeric(df_clean["TotalCharges"], errors="coerce")
df_clean["TotalCharges"] = df_clean["TotalCharges"].fillna(df_clean["TotalCharges"].median())

# limpando coluna de feedback do cliente
df_clean["CustomerFeedback"] = df_clean["CustomerFeedback"].fillna("").astype(str)
df_clean["CustomerFeedback_clean"] = (
    df_clean["CustomerFeedback"]
    .str.lower()
    .str.replace("[^a-zA-Z0-9 ]", "", regex=True)
)

# tratando categóricas com one-hot encoding
cat_cols = [
    "gender", "InternetService", "Contract", "PaymentMethod"
]

df_clean = pd.get_dummies(df_clean, columns=cat_cols, drop_first=True)

# convertendo target para binário
df_clean["Churn"] = df_clean["Churn"].map({"Yes": 1, "No": 0}).astype(int)


In [0]:
df_clean.info()


In [0]:
df_clean.head()

In [0]:
df_clean.to_csv("history_clean.csv", index=False)

In [0]:
import datetime

def prediction_function(input_df):
    '''
    An example model function, that just predicts randomly whether a customer will churn.
    TODO: Make a better model.
    '''
    X = input_df[['customerID']].copy()
    X['prediction'] = np.random.uniform(size=len(X)) >= 0.5
    X['prediction'] = X['prediction'].map({True: 'Yes', False: 'No'})
    return X

test_df = pd.read_csv('inference.csv')
prediction = prediction_function(test_df)
print(prediction.head().transpose())
# Use this code to save the prediction to a csv file for submission:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
prediction.to_csv(f'prediction_<MY_GROUP_NAME>_{timestamp}.csv')

In [0]:
plt.figure(figsize=(6,4))
sns.countplot(
    data=df_clean,
    x="Churn",
    hue="Churn",
    palette="Set2",
    legend=False
)
plt.title("Quantidade de Clientes: Churn vs Não Churn")
plt.xticks([0,1], ["Não churn", "Churn"])
plt.ylabel("Quantidade")
plt.xlabel("")
plt.show()


In [0]:
plt.figure(figsize=(10,5))
sns.histplot(
    data=df_clean,
    x="tenure",
    hue="Churn",
    multiple="stack",
    bins=40,
    palette="Set2"
)
plt.title("Distribuição de Tenure por Churn")
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.boxplot(
    data=df_clean,
    x="Churn",
    y="MonthlyCharges",
    hue="Churn",
    palette="Set2",
    legend=False   # evita legenda duplicada
)
plt.title("MonthlyCharges por Churn")
plt.xticks([0,1], ["Não churn", "Churn"])
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.violinplot(data=df_clean, x="Churn", y="TotalCharges", hue="Churn", palette="Set2", legend=False)
plt.title("Total Charges por Churn")
plt.show()


In [0]:
colors = ["#C0C0C0", "#2ECC71"]
# Reconstroi Contract
Contract_series = df_clean.apply(
    lambda r: (
        "Two year" if r["Contract_Two year"] == 1 else
        "One year" if r["Contract_One year"] == 1 else
        "Month-to-month"
    ),
    axis=1
)

# Reconstroi InternetService
InternetService_series = df_clean.apply(
    lambda r: (
        "Fiber optic" if r["InternetService_Fiber optic"] == 1 else
        "No internet" if r["InternetService_No"] == 1 else
        "DSL"
    ),
    axis=1
)

# Reconstroi PaymentMethod
PaymentMethod_series = df_clean.apply(
    lambda r: (
        "Credit card (automatic)" if r["PaymentMethod_Credit card (automatic)"] == 1 else
        "Electronic check" if r["PaymentMethod_Electronic check"] == 1 else
        "Mailed check" if r["PaymentMethod_Mailed check"] == 1 else
        "Bank transfer (automatic)"
    ),
    axis=1
)


In [0]:
def plot_stacked(series, churn, title, xlabel):
    ct = pd.crosstab(series, churn, normalize="index")
    ct.columns = ["No", "Yes"]
    ct.plot(kind="bar", stacked=True, figsize=(7,4), color=colors)
    plt.title(title)
    plt.ylabel("Proporção")
    plt.xlabel(xlabel)
    plt.ylim(0,1)
    plt.legend(title="Churn")
    plt.show()


In [0]:
plot_stacked(
    Contract_series,
    df_clean["Churn"],
    "Composição: Churn vs Não Churn — Contract",
    "Contract Type"
)

plot_stacked(
    InternetService_series,
    df_clean["Churn"],
    "Composição: Churn vs Não Churn — Internet Service",
    "Internet Service Type"
)
plot_stacked(
    PaymentMethod_series,
    df_clean["Churn"],
    "Composição: Churn vs Não Churn — Payment Method",
    "Payment Method"
)


In [0]:
plt.figure(figsize=(14,10))
corr = df_clean.corr(numeric_only=True)
sns.heatmap(corr, annot=False, cmap="coolwarm", linewidths=.5)
plt.title("Matriz de Correlação - Variáveis Numéricas")
plt.show()


In [0]:
ct_os = pd.crosstab(
    df_clean["OnlineSecurity"],
    df_clean["Churn"],
    normalize='index'
)
ct_os.columns = ["No", "Yes"]

ct_os.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn vs Não Churn — OnlineSecurity (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("OnlineSecurity")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
ct_ts = pd.crosstab(
    df_clean["TechSupport"],
    df_clean["Churn"],
    normalize='index'
)
ct_ts.columns = ["No", "Yes"]

ct_ts.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn vs Não Churn — TechSupport (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("TechSupport")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
ct_stv = pd.crosstab(
    df_clean["StreamingTV"],
    df_clean["Churn"],
    normalize='index'
)
ct_stv.columns = ["No", "Yes"]

ct_stv.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn vs Não Churn — StreamingTV (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("StreamingTV")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
ct_pb = pd.crosstab(
    df_clean["PaperlessBilling"],
    df_clean["Churn"],
    normalize='index'
)
ct_pb.columns = ["No", "Yes"]

ct_pb.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn — Paperless Billing (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("Paperless Billing")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


# Conclusões Completas da Análise Exploratória

## 1. Distribuição geral do churn
O gráfico de contagem mostra que a maioria dos clientes não churnou, com churn representando uma minoria relevante.  
Isso revela um desbalanceamento natural da variável-alvo, mas não impede a análise exploratória.

## 2. Relação entre tenure e churn
O histograma mostra um padrão claro:
- Clientes com tenure muito baixo apresentam alta incidência de churn.
- Conforme o tenure aumenta, a proporção de churn cai drasticamente.
- Clientes com tenure muito alto quase não apresentam churn.

Conclusão: churn é predominantemente concentrado nos clientes que recém entraram na base.

## 3. MonthlyCharges e churn
O boxplot indica:
- Clientes churn possuem MonthlyCharges mais altos.
- A mediana e o intervalo interquartil de churn são superiores aos de não churn.

Conclusão: mensalidades altas aumentam a probabilidade de churn.

## 4. TotalCharges e churn
O violin plot mostra:
- Clientes churn apresentam TotalCharges muito mais baixos.
- Clientes não churn concentram-se em valores altos de TotalCharges.

Conclusão: churn está fortemente associado a clientes com pouco tempo de relacionamento (TotalCharges baixo é um proxy de tenure baixo).

## 5. Tipo de Internet e churn (InternetService reconstruído)
O gráfico de proporção mostra três padrões:
- Fiber optic apresenta a maior proporção de churn.
- DSL tem churn moderado.
- No internet praticamente não tem churn.

Conclusão: o tipo de internet é um driver importante, com fibra óptica associada a maior insatisfação ou maior sensibilidade a preço.

## 6. Método de pagamento e churn
O gráfico de PaymentMethod indica:
- Clientes que utilizam Electronic check possuem proporção significativamente maior de churn.
- Os demais métodos apresentam churn bem menor.

Conclusão: Electronic check é um forte indicador de risco.

## 7. Contrato (Contract reconstruído)
No gráfico reconstruído:
- Month-to-month apresenta claramente a maior proporção de churn.
- One year e Two year exibem churn drasticamente menor.

Conclusão: contrato é um dos fatores de retenção mais relevantes: quanto mais longo, menor o churn.

## 8. OnlineSecurity e churn
O gráfico mostra:
- Clientes sem OnlineSecurity têm churn visivelmente maior.
- Clientes com OnlineSecurity churnam menos.

Conclusão: segurança adicional funciona como um mecanismo de retenção.

## 9. TechSupport e churn
O padrão é semelhante ao anterior:
- Ausência de TechSupport está associada a maior churn.
- Clientes com suporte apresentam taxas menores.

Conclusão: suporte técnico reduz churn, possivelmente por aumentar valor percebido.

## 10. StreamingTV e churn
O gráfico mostra:
- Pequena diferença de churn entre usar ou não usar StreamingTV.
- A variável não apresenta impacto forte.

Conclusão: StreamingTV é um fator secundário e não parece explicar churn de maneira contundente.

## 11. Heatmap de correlação
O heatmap confirma:
- As correlações de Pearson entre as variáveis numéricas e churn são baixas ou nulas.
- Isso é esperado, pois churn é uma variável binária com relações não lineares.

Conclusão: correlação linear não é apropriada para medir relação com churn; os gráficos categóricos capturam muito melhor os padrões.

---

# Síntese Geral dos Principais Fatores de Churn

## Fatores fortes (alta separação nos gráficos)
- Tenure baixo → churn alto.
- MonthlyCharges alto → maior churn.
- TotalCharges baixo → churn alto (clientes novos).
- InternetService = Fiber optic → churn elevado.
- PaymentMethod = Electronic check → churn elevado.
- Contract = Month-to-month → maior churn da base.
- Ausência de OnlineSecurity e TechSupport → aumento do churn.

## Fatores fracos
- StreamingTV (impacto pequeno).
- Outras variáveis numéricas não mostram relação linear significativa.
